<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.64
    Uninstalling yfinance-0.2.64:
      Successfully uninstalled yfinance-0.2.64
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-01 14:13:56
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.36 C
-------------------
Today PnL: -25.52 K (-0.19%)
Current PnL: -13.81 L (-9.62%)
CY Booked + Current PnL: -7.07 L (-4.93%)
-------------------
Total profit:  4.40 L
Total loss:  -18.21 L
-------------------
Total Booked + Current PnL: 19.91 L (17.08%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (4.95%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.08 C
Est FTT PnL: 72.24 L (53.08%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.58%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,UNIONBANK,123.87,163.00,1.69,M-LC,45.83,176167.0,35327.0,9161.0,0.87,25.08,5.20,31.59,89.0,3.86,1.30,53.03,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.00,2.03,M-LC,27.08,102630.0,24268.0,9760.0,-0.32,30.97,9.51,43.43,77.0,2.49,0.75,37.94,XY24,NTT,MISC
87,UJJIVANSFB,52.77,53.00,54.03,M-SC,89.58,133326.0,-9153.0,9773.0,0.71,-6.42,7.33,0.44,243.0,-0.94,0.98,57.36,OX40N,NTT,BANKS
39,HINDALCO,651.95,761.55,-19.90,H-LC,86.46,111000.0,6688.0,10845.0,0.13,6.41,9.77,16.81,27.0,0.62,0.82,23.49,X5K,ATH,METALS
71,SBIN,760.30,863.00,-10.56,M-LC,43.75,217459.0,15980.0,11243.0,0.03,7.93,5.17,13.51,76.0,1.42,1.60,21.53,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-29.63,L-SC,98.96,141236.0,-606.0,57172.0,1.91,-0.43,40.48,39.88,259.0,-0.01,1.04,43.13,XY24,NTT,POWER
63,QUESS,296.83,986.00,-33.88,M-SC,13.54,65580.0,574.0,150355.0,-0.86,0.88,229.27,232.18,191.0,0.00,0.48,0.42,XY24,NTT,MISC
55,LALPATHLAB,2804.19,3545.00,-33.57,H-SC,41.67,152762.0,-1468.0,42208.0,-0.67,-0.95,27.63,26.42,125.0,-0.03,1.12,20.47,X40N,NTT,HEALTHCARE
60,NATIONALUM,189.63,247.44,-50.15,H-MC,66.67,100912.0,408.0,30233.0,-1.19,0.41,29.96,30.49,97.0,0.01,0.74,32.94,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BLUESTARCO,1646.70,2337.55,-9.72,H-SC,85.42,169470.0,4800.0,64280.0,3.61,2.91,37.93,41.95,121.0,0.07,1.25,10.60,X40N,ATH,AC
5,APOLLOHOSP,6904.43,8285.00,-17.52,H-LC,79.17,112358.0,8792.0,11921.0,3.43,8.49,10.61,20.00,30.0,0.74,0.83,23.76,X40N,BTT,HEALTHCARE
79,SYMPHONY,1306.42,1306.00,-31.65,M-SC,6.25,145502.0,-25639.0,25579.0,3.30,-14.98,17.58,-0.03,189.0,-1.00,1.07,4.58,OX40N,NTT,DURABLES
54,KPIGREEN,497.21,732.16,29.50,H-SC,73.96,132401.0,7104.0,52100.0,2.09,5.67,39.35,47.25,139.0,0.14,0.97,65.64,MH,ATH,POWER
74,SIEMENS,4406.90,7969.85,-1.77,X-LC,42.71,82970.0,-27202.0,116274.0,2.07,-24.69,140.14,80.85,14.0,-0.23,0.61,22.20,AR,ATH,ELECTRICAL


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.55,NaN,NaN,2.08,16066.0,-33527.0,2450.0,-3.40,-67.60,15.25,-62.66,NaN,-13.68,0.12,7.74,XY24,ATH,MISC
29,DIXON,14367.00,18940.15,-36.85,H-MC,30.21,116192.0,1256.0,35334.0,-3.06,1.09,30.41,31.83,99.0,0.04,0.85,33.17,X40N,ATH,IT
4,ALKYLAMINE,2347.98,4546.37,6.76,X-SC,97.92,99394.0,-1569.0,96094.0,-2.39,-1.55,96.68,93.63,64.0,-0.02,0.73,48.05,SR,ATH,CHEMICALS
1,ABB,5551.76,7934.00,-35.10,H-LC,28.12,231524.0,15005.0,77908.0,-2.37,6.93,33.65,42.91,28.0,0.19,1.70,21.02,AR,NTT,ELECTRICAL
61,NESTLEIND,2268.60,2755.00,-11.07,X-LC,37.50,281970.0,16544.0,40378.0,-2.24,6.23,14.32,21.44,9.0,0.41,2.07,12.82,XY25,NTT,FMCG


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,54.03,M-SC,89.58,133326.0,-9153.0,9773.0,0.71,-6.42,7.33,0.44,243.0,-0.94,0.98,57.36,OX40N,NTT,BANKS
92,VIPIND,488.80,489.0,-215.66,H-SC,96.88,83421.0,-11895.0,11938.0,1.58,-12.48,14.31,0.04,154.0,-1.00,0.61,64.54,OX40N,NTT,MISC
79,SYMPHONY,1306.42,1306.0,-31.65,M-SC,6.25,145502.0,-25639.0,25579.0,3.30,-14.98,17.58,-0.03,189.0,-1.00,1.07,4.58,OX40N,NTT,DURABLES
81,TATAELXSI,7332.28,7332.0,-17.48,X-MC,19.79,86597.0,-16055.0,16055.0,-1.97,-15.64,18.54,-0.00,58.0,-1.00,0.64,31.64,OX40N,NTT,IT
46,INDIGOPNTS,1407.73,1408.0,123.21,H-SC,87.50,146444.0,-28115.0,28147.0,1.62,-16.11,19.22,0.02,127.0,-1.00,1.08,26.99,OX40N,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,1.04,42860.0,1605.0,9202.0,-0.22,3.89,21.47,26.19,1.0,0.17,0.32,0.00,XY24,ATH,IT
83,TCS,3794.03,4998.00,-18.06,X-LC,36.46,266744.0,-29190.0,123102.0,-1.22,-9.86,46.15,31.73,1.0,-0.24,1.96,6.77,X40,BTT,IT
48,INFY,1461.46,2275.00,-10.27,X-LC,59.38,287402.0,25801.0,119818.0,0.24,9.86,41.69,55.67,2.0,0.22,2.11,16.57,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-18.00,X-LC,20.83,256995.0,-13352.0,47981.0,-0.53,-4.94,18.67,12.81,6.0,-0.28,1.89,7.48,XY25,NTT,FMCG
61,NESTLEIND,2268.60,2755.00,-11.07,X-LC,37.50,281970.0,16544.0,40378.0,-2.24,6.23,14.32,21.44,9.0,0.41,2.07,12.82,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,UNIONBANK,123.87,163.0,1.69,M-LC,45.83,176167.0,35327.0,9161.0,0.87,25.08,5.20,31.59,89.0,3.86,1.30,53.03,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,2.03,M-LC,27.08,102630.0,24268.0,9760.0,-0.32,30.97,9.51,43.43,77.0,2.49,0.75,37.94,XY24,NTT,MISC
32,ENRIN,1377.95,2975.9,NaN,NaN,78.12,73752.0,39303.0,642.0,-0.87,114.09,0.87,115.97,NaN,61.22,0.54,12.78,AR,ATH,ELECTRICAL


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,91.10,M-MC,48.96,228923.0,3961.0,163703.0,0.72,1.76,71.51,74.53,185.0,0.02,1.68,32.91,XY24,BTT,STEEL
54,KPIGREEN,497.21,732.16,29.50,H-SC,73.96,132401.0,7104.0,52100.0,2.09,5.67,39.35,47.25,139.0,0.14,0.97,65.64,MH,ATH,POWER
20,CAMS,3643.00,5250.99,5.65,H-SC,53.12,118692.0,16688.0,28332.0,-1.19,16.36,23.87,44.14,124.0,0.59,0.87,35.26,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,2.03,M-LC,27.08,102630.0,24268.0,9760.0,-0.32,30.97,9.51,43.43,77.0,2.49,0.75,37.94,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,1.69,M-LC,45.83,176167.0,35327.0,9161.0,0.87,25.08,5.20,31.59,89.0,3.86,1.30,53.03,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,78.12,73752.0,39303.0,642.0,-0.87,114.09,0.87,115.97,NaN,61.22,0.54,12.78,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-38.56,M-SC,91.67,202537.0,30916.0,131163.0,-1.68,18.01,64.76,94.43,198.0,0.24,1.49,49.48,XR,NTT,DURABLES
54,KPIGREEN,497.21,732.16,29.50,H-SC,73.96,132401.0,7104.0,52100.0,2.09,5.67,39.35,47.25,139.0,0.14,0.97,65.64,MH,ATH,POWER
95,WIPRO,243.46,420.00,-4.22,M-LC,71.88,163928.0,12983.0,96472.0,-0.59,8.60,58.85,72.51,69.0,0.13,1.21,12.92,XR,NTT,IT
45,INDIAMART,2327.09,4871.06,-47.33,H-SC,92.71,137005.0,13669.0,121167.0,-0.50,11.08,88.44,109.32,122.0,0.11,1.01,35.35,AR,ATH,MISC


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-51.71,L-SC,100.00,70963.0,-42586.0,82636.0,-1.31,-37.50,116.45,35.27,261.0,-0.52,0.52,73.68,XR,NTT,HOTELS
32,ENRIN,1377.95,2975.90,NaN,NaN,78.12,73752.0,39303.0,642.0,-0.87,114.09,0.87,115.97,NaN,61.22,0.54,12.78,AR,ATH,ELECTRICAL
58,MASFIN,326.60,402.39,-18.25,H-SC,63.54,92700.0,-5280.0,28014.0,0.31,-5.39,30.22,23.21,137.0,-0.19,0.68,34.45,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,6.76,X-SC,97.92,99394.0,-1569.0,96094.0,-2.39,-1.55,96.68,93.63,64.0,-0.02,0.73,48.05,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,2.03,M-LC,27.08,102630.0,24268.0,9760.0,-0.32,30.97,9.51,43.43,77.0,2.49,0.75,37.94,XY24,NTT,MISC


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-51.71,L-SC,100.00,70963.0,-42586.0,82636.0,-1.31,-37.50,116.45,35.27,261.0,-0.52,0.52,73.68,XR,NTT,HOTELS
52,JPPOWER,18.73,26.20,-29.63,L-SC,98.96,141236.0,-606.0,57172.0,1.91,-0.43,40.48,39.88,259.0,-0.01,1.04,43.13,XY24,NTT,POWER
4,ALKYLAMINE,2347.98,4546.37,6.76,X-SC,97.92,99394.0,-1569.0,96094.0,-2.39,-1.55,96.68,93.63,64.0,-0.02,0.73,48.05,SR,ATH,CHEMICALS
92,VIPIND,488.80,489.00,-215.66,H-SC,96.88,83421.0,-11895.0,11938.0,1.58,-12.48,14.31,0.04,154.0,-1.00,0.61,64.54,OX40N,NTT,MISC
51,JIOFIN,310.58,387.00,7.86,H-LC,95.83,231105.0,13699.0,39796.0,1.04,6.30,17.22,24.61,95.0,0.34,1.70,64.32,XY24,BTT,FINANCE


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.82
1,25,41.89
2,50,70.42


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.51
LC    34.11
MC    21.38
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.53
X40      12.23
XY25     12.02
X40N     11.80
XR       10.13
AR        8.26
OX40N     8.20
MH        1.71
X5K       1.48
X200      1.40
SR        1.22
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.65
M-SC    16.02
H-LC    14.18
X-LC    12.19
H-MC     9.46
X-MC     7.50
M-LC     6.68
X-SC     4.36
M-MC     4.05
L-SC     2.48
L-LC     1.06
L-MC     0.37
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.10,-1.95,36.04
IT,12.05,-10.82,65.24
BANKS,7.99,-4.98,47.82
FINANCE,7.94,-11.52,51.89
MISC,6.62,-15.20,68.65
PAINTS,5.68,-12.99,35.26
ELECTRICAL,5.48,1.17,44.98
HEALTHCARE,4.52,-4.84,35.47
AUTO,4.30,-15.38,66.11


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2714275.0,27
XR,1080958.0,13
AR,1020513.0,10
X40,655398.0,9
X40N,559948.0,11
XY25,483395.0,8
OX40N,313798.0,11
SR,177509.0,2
MH,82333.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1973068.0,20
M-SC,1765282.0,21
H-LC,611112.0,12
H-MC,546400.0,8
X-LC,516315.0,7
X-MC,433032.0,6
M-MC,419007.0,3
X-SC,324305.0,5
M-LC,265557.0,5


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      861917.0      6
M-SC       XY24      814749.0      7
H-SC       AR        520133.0      3
M-SC       XR        336057.0      4
M-MC       XY24      309126.0      2
H-MC       XY24      298796.0      3
X-LC       X40       242920.0      2
H-SC       X40N      229860.0      4
           XR        218616.0      3
H-LC       X40N      199784.0      4
M-SC       AR        182082.0      2
           OX40N     167132.0      5
M-LC       XY24      157842.0      3
H-LC       X40       157692.0      3
X-MC       X40       153188.0      2
L-SC       XR        144153.0      2
X-SC       XR        123587.0      1
X-LC       XY25      120671.0      3
           AR        116274.0      1
M-SC       XY25      111621.0      1
M-MC       XR        109881.0      1
X-MC       XY25      106499.0      1
           XY24       98770.0      1
M-LC       XR         96472.0      1
X-SC       SR         96094.0      1
H-SC       OX40N      90442.0      3
H-MC       XY25       88551.0      1
M-SC       SR         81415.0      1
H-LC       AR         77908.0      1
M-SC       X40        72226.0      1
H-LC       X200       69294.0      1
           X5K        66638.0      2
X-SC       XY24       64455.0      1
H-MC       AR         64114.0      1
L-SC       AR         59360.0      1
X-MC       X40N       58520.0      1
L-SC       XY24       57172.0      1
L-MC       XR         52192.0      1
H-SC       MH         52100.0      1
L-LC       XY25       44810.0      1
X-SC       OX40N      40169.0      2
H-LC       XY24       39796.0      1
X-LC       X40N       36450.0      1
H-MC       X40N       35334.0      1
           MH         30233.0      1
           X40        29372.0      1
X-MC       OX40N      16055.0      1
M-LC       XY25       11243.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
